## Create .csv File to Use For Shiny App

Once the decision was made to use a rules based model, a .csv file was put together that only had the necessary information for the shiny app. When making the Shiny App it was found that missing values were causing problems. Schools with missing values in the key variables were dropped from the final .csv and app. The total of schools missing enrollment data was one, schools missing data on absenteeism 18, schools missing data on days missed due to suspensions 19, and schools without data on sports participation was 3,461. The final .csv file had 63 columns.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math

In [2]:
numbers = pd.read_csv('/Users/flatironschool/Absenteeism_Project/data/processed/combo_cleaned.csv')


In [3]:
numbers.tail()

,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,...,sports_part,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate
21862,21862,21862,WYOMING,Washakie County School District #2,560582000393,Ten Sleep K-12,10,GE50,WY,WYOMING,...,31.0,0.281818,0.0,0.000000,0.0,0.0,4407.512636,0.0,0.009091,0.000000
21863,21863,21863,WYOMING,Teton County School District #1,560583000335,Jackson Hole High School,127,GE95,WY,WYOMING,...,355.0,0.554688,26.0,0.040625,0.0,0.0,4153.518984,0.0,0.004687,0.118939
21864,21864,21864,WYOMING,Teton County School District #1,560583000512,Summit High School,18,GE80,WY,WYOMING,...,NaN,NaN,10.0,0.185185,0.0,0.0,5469.872593,0.0,0.000000,0.091491
21865,21865,21865,WYOMING,Weston County School District #7,560609000401,Upton High School,21,GE80,WY,WYOMING,...,85.0,0.850000,0.0,0.000000,0.0,0.0,6875.370000,0.0,0.000000,0.162162
21866,21866,21866,WYOMING,Washakie County School District #1,560624000343,Worland High School,105,75-79,WY,WYOMING,...,NaN,NaN,10.0,0.025000,0.0,0.0,3015.755325,0.0,0.004275,0.331544


## Clean up graduation rates and add grad rate bins

In [4]:
#need to keep original reported grade and need column to modify and clean data
numbers['grad_slice'] = numbers['ALL_RATE_1516']

In [5]:
#remove "GE" and "LE" from ranges
numbers['grad_slice'].replace(['GE99'], '+100', inplace=True) # need to replace with 
numbers['grad_slice'].replace(['GE95'], '95', inplace=True)
numbers['grad_slice'].replace(['GE90'], '90', inplace=True)
numbers['grad_slice'].replace(['LE10'], '10', inplace=True)
numbers['grad_slice'].replace(['LE1'], '1', inplace=True)
numbers['grad_slice'].replace(['LE5'], '05', inplace=True)


In [6]:
#smallest range needs to be dealt with, has one digit before '-'
numbers['grad_slice'].replace(['6-9'], '6', inplace=True)

In [7]:
#take first two digits of rates to get rid of ranges
numbers['grad_slice'] = numbers['grad_slice'].str[:2]

In [8]:
#fix 100
numbers['grad_slice'].replace(['+1'], '100', inplace=True)

In [9]:
#get rid of very small schools
grad_num = numbers[numbers['ALL_COHORT_1516'] >= 31]

In [10]:
#create the binned categories
grad_num['grad_rate_bin'] = pd.cut(grad_num['grad_slice'].astype(int), [0, 59, 79, 89, 99, 100],
      labels = ['0-59%', '60-79%', '80-89%', '90-99%', '100%'])

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
grad_num.tail()

,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,...,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice,grad_rate_bin
21854,21854,21854,WYOMING,Sheridan County School District #1,560569000311,Big Horn High School,37,GE90,WY,WYOMING,...,2.0,0.013245,4.0,0.026490,1738.913907,0.0,0.006623,0.846154,90,90-99%
21858,21858,21858,WYOMING,Sheridan County School District #2,560569500360,Sheridan High School,236,89,WY,WYOMING,...,46.0,0.046796,0.0,0.000000,1694.559176,0.0,0.005086,0.198779,89,80-89%
21861,21861,21861,WYOMING,Sweetwater County School District #2,560576200324,Green River High School,176,85-89,WY,WYOMING,...,110.0,0.141570,1.0,0.001287,1840.877490,0.0,0.003861,0.200000,85,80-89%
21863,21863,21863,WYOMING,Teton County School District #1,560583000335,Jackson Hole High School,127,GE95,WY,WYOMING,...,26.0,0.040625,0.0,0.000000,4153.518984,0.0,0.004687,0.118939,95,90-99%
21866,21866,21866,WYOMING,Washakie County School District #1,560624000343,Worland High School,105,75-79,WY,WYOMING,...,10.0,0.025000,0.0,0.000000,3015.755325,0.0,0.004275,0.331544,75,60-79%


In [12]:
grad_num['grad_rate_bin'].value_counts()

90-99%    8484
80-89%    4257
60-79%    1993
0-59%     1611
100%       228
Name: grad_rate_bin, dtype: int64

## Create Level Up Bins 

In [13]:
#level up bins
#create the binned categories
grad_num['level_up_bins'] = pd.cut(grad_num['grad_slice'].astype(int), 
    [0, 59, 79, 89, 99, 100], labels = ['60-79% Level Up Rate', '80-89% Level Up Rate', '90-99% Level Up Rate', '100% Level Up Rate', '100% Top Rate'])

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [14]:
grad_num.head()

,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,...,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice,grad_rate_bin,level_up_bins
0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,ALABAMA,...,0.060232,0.0,0.000000,2811.937359,0.0,0.003475,0.378788,92,90-99%,100% Level Up Rate
1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,GE95,AL,ALABAMA,...,0.018553,2.0,0.003711,4825.189777,0.0,0.002783,0.100000,95,90-99%,100% Level Up Rate
2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,AL,ALABAMA,...,0.030303,5.0,0.008418,5317.932795,0.0,0.001684,0.105263,85,80-89%,90-99% Level Up Rate
3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,AL,ALABAMA,...,0.021786,0.0,0.000000,5909.375686,0.0,0.002179,0.068966,80,80-89%,90-99% Level Up Rate
4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,AL,ALABAMA,...,0.022039,0.0,0.000000,3962.305785,0.0,0.002755,0.101010,85,80-89%,90-99% Level Up Rate


## Calculate Quantiles and Add to Data Frame

In [15]:
quantile_df_25 = grad_num.groupby('grad_rate_bin')['non_cert_rate', 'sports_rate', 'chronic_absent_rate', 'suspensed_day_rate'].quantile(.25).reset_index() 

In [16]:
quantile_df_75 = grad_num.groupby('grad_rate_bin')['non_cert_rate', 'sports_rate', 'chronic_absent_rate', 'suspensed_day_rate'].quantile(.75).reset_index() 

In [17]:
quantile_df_25.head()

0.25,grad_rate_bin,non_cert_rate,sports_rate,chronic_absent_rate,suspensed_day_rate
0,0-59%,0.0,0.124414,0.085595,0.000000
1,60-79%,0.0,0.220779,0.160080,0.060664
2,80-89%,0.0,0.275554,0.126976,0.052518
3,90-99%,0.0,0.319483,0.084154,0.016064
4,100%,0.0,0.364777,0.059735,0.008866


In [18]:
quantile_df_75.head()

0.75,grad_rate_bin,non_cert_rate,sports_rate,chronic_absent_rate,suspensed_day_rate
0,0-59%,0.042020,0.406508,0.790489,0.503487
1,60-79%,0.052632,0.451950,0.437500,0.672138
2,80-89%,0.023318,0.536686,0.302113,0.351397
3,90-99%,0.000000,0.649428,0.223554,0.148475
4,100%,0.012500,0.651462,0.143481,0.061694


In [19]:
grad_num = grad_num.merge(quantile_df_25, on='grad_rate_bin', suffixes=('_x', '_25th'))

In [22]:
grad_num = grad_num.merge(quantile_df_75, on='grad_rate_bin', suffixes=('_y', '_75th'))

In [23]:
grad_num.head()

,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,...,chronic_absent_rate_25th,suspensed_day_rate_25th,non_cert_rate_y,sports_rate_y,chronic_absent_rate_y,suspensed_day_rate_y,non_cert_rate_75th,sports_rate_75th,chronic_absent_rate_75th,suspensed_day_rate_75th
0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,ALABAMA,...,0.084154,0.016064,0.0,0.649428,0.223554,0.148475,0.0,0.649428,0.223554,0.148475
1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,GE95,AL,ALABAMA,...,0.084154,0.016064,0.0,0.649428,0.223554,0.148475,0.0,0.649428,0.223554,0.148475
2,5,5,ALABAMA,Hoover City,10000700251,Hoover High Sch,714,92,AL,ALABAMA,...,0.084154,0.016064,0.0,0.649428,0.223554,0.148475,0.0,0.649428,0.223554,0.148475
3,7,7,ALABAMA,Hoover City,10000701456,Spain Park High Sch,412,94,AL,ALABAMA,...,0.084154,0.016064,0.0,0.649428,0.223554,0.148475,0.0,0.649428,0.223554,0.148475
4,8,8,ALABAMA,Madison City,10000800831,Bob Jones High Sch,451,97,AL,ALABAMA,...,0.084154,0.016064,0.0,0.649428,0.223554,0.148475,0.0,0.649428,0.223554,0.148475


## Change column names to final names for shiny app

In [25]:
grad_num.rename(columns={'total_enrollment':'Total_Enrollment','total_chronic_absent':'Number_of_Chronically_Absent_Students', 'sports_part':'Number_of_Student_Athletes'},inplace=True)
grad_num.rename(columns={'ALL_RATE_1516':'Graduation_Rate_2015_16'}, inplace=True)
grad_num.rename(columns={'SCH_FTETEACH_NOTCERT':'Number_of_Non_Certified_Teachers'}, inplace=True)
grad_num.rename(columns={'level_up_bins':'Level_Up_Graduation_Rate'}, inplace=True)
grad_num.rename(columns={'STNAM':'State', 'LEANM':'District', 'SCHNAM':'High_School', 'ALL_RATE_1516':'Graduation_Rate_2015_16'},inplace=True)


## Calculate Middle 50% Range for App

In [26]:
grad_num['Level_Up_25th_Percentile_Number_Chronic_Absent_Students'] = grad_num['Total_Enrollment'] * grad_num['chronic_absent_rate_25th']
grad_num['Level_Up_75th_Percentile_Number_Chronic_Absent_Students'] = round(grad_num['Total_Enrollment'] * grad_num['chronic_absent_rate_75th'],0)
grad_num['Level_Up_25th_Percentile_Student_Athletes'] = round(grad_num['Total_Enrollment'] * grad_num['sports_rate_25th'],0)
grad_num['Level_Up_75th_Percentile_Student_Athletes'] = round(grad_num['Total_Enrollment'] * grad_num['sports_rate_75th'],0)
grad_num['Level_Up_25th_Percentile_Days_Missed_due_to_Suspension'] = round(grad_num['Total_Enrollment'] * grad_num['suspensed_day_rate_25th'],0)
grad_num['Level_Up_75th_Percentile_Days_Missed_due_to_Suspension'] = round(grad_num['Total_Enrollment'] * grad_num['suspensed_day_rate_75th'],0)
grad_num['Level_Up_25th_Percentile_Non_Certified_Teachers'] = round(grad_num['Total_Enrollment'] * grad_num['non_cert_rate_25th'],0)
grad_num['Level_up_75th_Percentile_Non_Certified_Teachers'] = round(grad_num['Total_Enrollment'] * grad_num['non_cert_rate_75th'],0)

In [27]:
grad_num.head()

,Unnamed: 0,Unnamed: 0.1,State,District,NCESSCH,High_School,ALL_COHORT_1516,Graduation_Rate_2015_16,LEA_STATE,LEA_STATE_NAME,...,chronic_absent_rate_75th,suspensed_day_rate_75th,Level_Up_25th_Percentile_Number_Chronic_Absent_Students,Level_Up_75th_Percentile_Number_Chronic_Absent_Students,Level_Up_25th_Percentile_Student_Athletes,Level_Up_75th_Percentile_Student_Athletes,Level_Up_25th_Percentile_Days_Missed_due_to_Suspension,Level_Up_75th_Percentile_Days_Missed_due_to_Suspension,Level_Up_25th_Percentile_Non_Certified_Teachers,Level_up_75th_Percentile_Non_Certified_Teachers
0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,ALABAMA,...,0.223554,0.148475,108.979716,290.0,414.0,841.0,21.0,192.0,0.0,0.0
1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,GE95,AL,ALABAMA,...,0.223554,0.148475,45.359125,120.0,172.0,350.0,9.0,80.0,0.0,0.0
2,5,5,ALABAMA,Hoover City,10000700251,Hoover High Sch,714,92,AL,ALABAMA,...,0.223554,0.148475,249.685574,663.0,948.0,1927.0,48.0,441.0,0.0,0.0
3,7,7,ALABAMA,Hoover City,10000701456,Spain Park High Sch,412,94,AL,ALABAMA,...,0.223554,0.148475,141.126629,375.0,536.0,1089.0,27.0,249.0,0.0,0.0
4,8,8,ALABAMA,Madison City,10000800831,Bob Jones High Sch,451,97,AL,ALABAMA,...,0.223554,0.148475,147.774812,393.0,561.0,1140.0,28.0,261.0,0.0,0.0


## Add back in the Chronic Absentee Rate and Sports Participation Rate for each school

In [39]:
grad_num['Chronic_Absent_Rate'] = grad_num['Number_of_Chronically_Absent_Students']/grad_num['Total_Enrollment']
grad_num['Sports_Participant_Rate'] = grad_num['Number_of_Student_Athletes']/grad_num['Total_Enrollment']

## Clean up final data frame and save to csv

In [29]:
grad_num.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
grad_num.drop(['LEA_STATE_NAME', 'SCH_NAME', 'SCH_MAGNETDETAIL','SCH_ALTFOCUS', 'TOT_GTENR_M', 'TOT_GTENR_F'], axis=1, inplace=True)
grad_num.drop(grad_num.columns.to_series()['TOT_ALGENR_GS0910_M':'TOT_SATACT_F'], axis=1, inplace=True)
grad_num.drop(grad_num.columns.to_series()['SCH_HBALLEGATIONS_SEX':'SCH_HBALLEGATIONS_REL'], axis=1, inplace=True)
grad_num.drop(['SCH_NPE_WOFED', 'SCH_NPE_WFED', 'SCH_FTECOUNSELORS', 'SCH_FTETEACH_ABSENT'], axis=1, inplace=True)
grad_num.drop(grad_num.columns.to_series()['total_ap_ib_de':'calc_rate'], axis=1, inplace=True)
grad_num.drop(grad_num.columns.to_series()['harassed':'activities_funds_rate'], axis=1, inplace=True)
grad_num.drop(['counselor_rate', 'absent_teacher_rate'], axis=1, inplace=True)

In [37]:
grad_num.drop(['TOT_DUAL_M', 'TOT_DUAL_F'], axis=1, inplace=True)

In [38]:
#delete rows no longer needed for shiny app
#delete rows with NANs
#no missing values
grad_num.isna().sum()

State                                                      0
District                                                   0
NCESSCH                                                    0
High_School                                                0
ALL_COHORT_1516                                            0
Graduation_Rate_2015_16                                    0
LEA_STATE                                                  0
LEAID_y                                                    0
LEA_NAME                                                   0
SCHID                                                      0
COMBOKEY                                                   0
JJ                                                         0
SCH_STATUS_SPED                                            0
SCH_STATUS_MAGNET                                          0
SCH_STATUS_CHARTER                                         0
SCH_STATUS_ALT                                             0
TOT_ENR_M               

In [31]:
#delete rows with NANs in enrollment, absenteeism, sports participation, non-certified teachers,
#and days missed to suspension
grad_num.dropna(subset=['Total_Enrollment', 'TOT_ENR_M', 'TOT_ENR_F'], inplace=True)

In [32]:
grad_num.dropna(subset=['Number_of_Chronically_Absent_Students','TOT_ABSENT_M', 'TOT_ABSENT_F'], inplace=True)

In [33]:
grad_num.dropna(subset=['Number_of_Student_Athletes', 'SCH_SSPART_M', 'SCH_SSPART_F', 'TOT_SSPART'], inplace=True)

In [34]:
grad_num.dropna(subset=['Number_of_Non_Certified_Teachers', 'non_cert_rate_x', 'SCH_FTETEACH_TOT'], inplace=True)

In [35]:
grad_num.dropna(subset=['TOT_DAYSMISSED_M', 'TOT_DAYSMISSED_F', 'total_suspension_days', 'suspensed_day_rate_x'], inplace=True)
                

## Save the final data frame to csv for future use

In [40]:
grad_num2 = grad_num

In [41]:
#data frame deleting NANs
grad_num2.to_csv('grad_num2.csv')

In [42]:
grad_num2.head()

,State,District,NCESSCH,High_School,ALL_COHORT_1516,Graduation_Rate_2015_16,LEA_STATE,LEAID_y,LEA_NAME,SCHID,...,Level_Up_25th_Percentile_Number_Chronic_Absent_Students,Level_Up_75th_Percentile_Number_Chronic_Absent_Students,Level_Up_25th_Percentile_Student_Athletes,Level_Up_75th_Percentile_Student_Athletes,Level_Up_25th_Percentile_Days_Missed_due_to_Suspension,Level_Up_75th_Percentile_Days_Missed_due_to_Suspension,Level_Up_25th_Percentile_Non_Certified_Teachers,Level_up_75th_Percentile_Non_Certified_Teachers,Chronic_Absent_Rate,Sports_Participant_Rate
0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,100005,Albertville City,871,...,108.979716,290.0,414.0,841.0,21.0,192.0,0.0,0.0,0.213127,0.169884
1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,GE95,AL,100006,Marshall County,872,...,45.359125,120.0,172.0,350.0,9.0,80.0,0.0,0.0,0.202226,0.324675
2,ALABAMA,Hoover City,10000700251,Hoover High Sch,714,92,AL,100007,Hoover City,251,...,249.685574,663.0,948.0,1927.0,48.0,441.0,0.0,0.0,0.144253,0.302663
3,ALABAMA,Hoover City,10000701456,Spain Park High Sch,412,94,AL,100007,Hoover City,1456,...,141.126629,375.0,536.0,1089.0,27.0,249.0,0.0,0.0,0.171139,0.313655
4,ALABAMA,Madison City,10000800831,Bob Jones High Sch,451,97,AL,100008,Madison City,831,...,147.774812,393.0,561.0,1140.0,28.0,261.0,0.0,0.0,0.009112,0.321754
